In [95]:
import pinocchio as pin 
import numpy as np
import time
from scipy.optimize import fmin_bfgs,fmin_slsqp
from numpy.linalg import pinv,inv,norm,svd,eig
from tools import collision, getcubeplacement, setcubeplacement, projecttojointlimits
from config import LEFT_HOOK, RIGHT_HOOK, LEFT_HAND, RIGHT_HAND, EPSILON
from config import CUBE_PLACEMENT, CUBE_PLACEMENT_TARGET

from tools import setcubeplacement

In [142]:
def cost(q):
    oMcube  = getcubeplacement(cube) #origin of the cube
    oMcubeL = getcubeplacement(cube, LEFT_HOOK) #placement of the left hand hook
    oMcubeR = getcubeplacement(cube, RIGHT_HOOK) #placement of the right hand hook
    
    pin.framesForwardKinematics(robot.model,robot.data,q)   
    pin.computeJointJacobians(robot.model,robot.data,q)
    
    frameidL = robot.model.getFrameId(LEFT_HAND)
    oMframeL = robot.data.oMf[frameidL]
    frameidR = robot.model.getFrameId(RIGHT_HAND)
    oMframeR = robot.data.oMf[frameidR]
    
    L_hand_tran = oMframeL.translation - oMcubeL.translation
    R_hand_tran = oMframeR.translation - oMcubeR.translation
    L_hand_rot = oMframeL.rotation - oMcubeL.rotation
    R_hand_rot = oMframeR.rotation - oMcubeR.rotation
    
    #hand_tran = oMframeL.translation
    #hook_tran = oMcubeL.translation
    #hand_rot = oMframeL.rotation
    #hook_rot = oMcubeL.rotation
    L_hand_difference = np.linalg.norm(L_hand_tran)**2 + np.linalg.norm(L_hand_rot)**2
    R_hand_difference = np.linalg.norm(R_hand_tran)**2 + np.linalg.norm(R_hand_rot)**2
    
    return R_hand_difference + L_hand_difference

In [143]:
def callback(q):
    updatevisuals(viz, robot, cube, q)
    time.sleep(.1)

In [144]:
# %load tp2/generated/configuration_reduced_optim
q = robot.q0.copy() 
qopt_bfgs = fmin_bfgs(cost,q, callback=callback)
print('\n *** Optimal configuration from BFGS = %s \n\n\n\n' % qopt_bfgs)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 34
         Function evaluations: 624
         Gradient evaluations: 39

 *** Optimal configuration from BFGS = [-3.35219172e-01  0.00000000e+00  0.00000000e+00 -9.24865003e-01
 -2.13362242e-02 -2.17579274e-01  3.11936244e-07  2.38916007e-01
 -2.39916259e-01 -2.18930331e-01 -2.99015137e-01  9.37001871e-02
  7.63676000e-07  2.05315748e-01  2.19733403e+00] 






In [134]:
def computeqgrasppose(robot, qcurrent, cube, cubetarget, viz=None):
    '''Return a collision free configuration grasping a cube at a specific location and a success flag'''
    setcubeplacement(robot, cube, cubetarget)
    
    # %load tp2/generated/configuration_reduced_optim
    q = robot.q0.copy() 
    qopt_bfgs = fmin_bfgs(cost,q, callback=callback)
    print('\n *** Optimal configuration from BFGS = %s \n\n\n\n' % qopt_bfgs)


    return qopt_bfgs, False
            

In [145]:
if __name__ == "__main__":
    from tools import setupwithmeshcat
    from setup_meshcat import updatevisuals
    robot, cube, viz = setupwithmeshcat(url="tcp://127.0.0.1:6004")
    
    q = robot.q0.copy()
    
    qe,successend = computeqgrasppose(robot, q, cube, CUBE_PLACEMENT_TARGET,  viz)
    q0, successinit = computeqgrasppose(robot, q, cube, CUBE_PLACEMENT, viz)
    
    updatevisuals(viz, robot, cube, qe)
    
    

Wrapper tries to connect to server <tcp://127.0.0.1:6004>
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7004/static/
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 33
         Function evaluations: 592
         Gradient evaluations: 37

 *** Optimal configuration from BFGS = [ 1.27001940e-01  0.00000000e+00  0.00000000e+00 -1.51057818e-01
  3.21841913e-02 -2.70865357e-01  1.41538747e-06  2.38680038e-01
 -1.47594526e+00  4.89201687e-01  1.14989471e-01 -3.49028086e-01
 -2.79294207e-08  2.34037898e-01  1.02698129e+00] 




Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 34
         Function evaluations: 624
         Gradient evaluations: 39

 *** Optimal configuration from BFGS = [-3.35219172e-01  0.00000000e+00  0.00000000e+00 -9.24865003e-01
 -2.13362242e-02 -2.17579274e-01  3.11936244e-07  2.38916007e-01
 -2.39916259e-01 -2.18930331e-01 -2.99015137e-0

In [136]:
from config import LEFT_HOOK, RIGHT_HOOK, CUBE_PLACEMENT, CUBE_PLACEMENT_TARGET

from tools import getcubeplacement, setcubeplacement
from setup_meshcat import updatevisuals

#We can access the current cube position using
oMcube  = getcubeplacement(cube) #origin of the cube
oMcubeL = getcubeplacement(cube, LEFT_HOOK) #placement of the left hand hook
oMcubeR = getcubeplacement(cube, RIGHT_HOOK) #placement of the right hand hook

#the cube position is updated using the following function:
setcubeplacement(robot, cube, CUBE_PLACEMENT)
#to update the frames for both the robot and the cube you can call
updatevisuals(viz, robot, cube, q)


In [137]:
    oMcube  = getcubeplacement(cube) #origin of the cube
    oMcubeL = getcubeplacement(cube, LEFT_HOOK) #placement of the left hand hook
    oMcubeR = getcubeplacement(cube, RIGHT_HOOK) #placement of the right hand hook
    #update the frame positions in robot.data given q
    pin.framesForwardKinematics(robot.model,robot.data,q)   
    pin.computeJointJacobians(robot.model,robot.data,q)
    frameidL = robot.model.getFrameId(LEFT_HAND)
    frameidR = robot.model.getFrameId(RIGHT_HAND)   
    oMframeL = robot.data.oMf[frameidL] 
    oMframeR = robot.data.oMf[frameidR]
    
    print(oMcubeL)
    #print(oMcubeR.translation)
    print(oMframeL)
    #print(oMframeR.translation)

  R =
1 0 0
0 1 0
0 0 1
  p =  0.33 -0.25  0.93

  R =
0.0707372 -0.997495         0
 0.997495 0.0707372         0
        0         0         1
  p =  0.45  0.28 0.851

